In [28]:
import pandas as pd
import re

# --- Caricamento file ---
eucs = pd.read_csv("Other/EUCS PDF official file/EUCS_AnnexA_Extraction_Perfect.csv")
medina = pd.read_csv("Schemes/MEDINA_EUCS_ControlMapping_Clean.csv")

In [29]:
eucs

,controlId,description,assurance_level,baseId
0,OIS-01.1,"The CSP shall define, implement, maintain and ...",Basic,OIS-01
1,OIS-01.3,The ISMS shall have a valid certification acco...,High,OIS-01
2,OIS-01.4,The CSP shall document the measures for docume...,Basic,OIS-01
3,OIS-01.5,The documentation shall include at least: - Sc...,Substantial,OIS-01
4,OIS-02.1,The CSP shall perform a risk assessment as def...,Basic,OIS-02
...,...,...,...,...
462,PSS-04.1,The CSP shall ensure the following aspects if ...,Basic,PSS-04
463,PSS-04.2,The CSP shall ensure the following aspects if ...,Substantial,PSS-04
464,PSS-04.3,An integrity check shall be performed and auto...,High,PSS-04
465,PSS-05.1,The CSP shall allow the CSC to specify the loc...,Substantial,PSS-05


In [30]:
medina

,EUCS Category,EUCS Control (2022),EUCS Text,Code,C5.2020 GERMANY,SecNumCloud FRANCE,ISO 27002,ISO 27017
0,A1 - Organisation of Information Security,OIS-01 - INFORMATION SECURITY MANAGEMENT SYSTEM,NaN,NaN,OIS-01,NaN,5.4\n5.21,NaN
1,NaN,OIS-02 - SEGREGATION OF DUTIES,NaN,OIS-02,OIS-04,6.1\n 6.2,5.3,CLD.6.3.1
2,NaN,OIS-03 - CONTACT WITH AUTHORITIES AND INTERES...,NaN,NaN,OIS-05,6.3\n 6.4,5.5\n5.6,CLD.6.3.1
3,NaN,OIS-04 - INFORMATION SECURITY IN PROJECT MANAG...,NaN,NaN,OIS-05\nOIS-6\nOIS-7,6.5,5.8,CLD.6.3.1
4,A2 - Information Security Policies,ISP-01 - GLOBAL INFORMATION SECURITY POLICY,NaN,NaN,OIS-02,5.2\n 5.1,5.1,NaN
...,...,...,...,...,...,...,...,...
115,A20 - Product Security,PSS-01 - ERROR HANDLING AND LOGGING MECHANISMS,NaN,NaN,PSS-04,NaN,8.15,NaN
116,NaN,PSS-02 - SESSION MANAGEMENT,NaN,NaN,PSS-06,NaN,8.16,NaN
117,NaN,PSS-03 - SOFTWARE DEFINED NETWORKING,NaN,NaN,PSS-10,NaN,8.20\n8.21,CLD.13.1.4
118,NaN,PSS-04 - IMAGES FOR VIRTUAL MACHINES AND CONT...,NaN,PSS-04,PSS-11,NaN,NaN,NaN


In [31]:
import pandas as pd
import re

# --- 1. CONFIGURAZIONE E CARICAMENTO ---
# (Uso i dataframe che hai già caricato nell'esempio, supponendo si chiamino 'eucs' e 'medina')
# Se devi ricaricarli, decommenta le righe sotto:
# eucs = pd.read_csv("Other/EUCS PDF official file/EUCS_AnnexA_Extraction.csv")
# medina = pd.read_csv("Schemes/MEDINA_EUCS_ControlMapping_Clean.csv")

print("--- INIZIO MERGE (VERSIONE PULITA) ---")
print(f"Righe originali EUCS: {len(eucs)}")
print(f"Righe originali MEDINA: {len(medina)}")

# --- STEP AGGIUNTO: PULIZIA PRELIMINARE ---
# Rimuoviamo le righe dal file di mapping che non hanno un ID di controllo (le righe "sporche")
medina = medina.dropna(subset=['EUCS Control (2022)'])
# Rimuoviamo anche se sono stringhe vuote
medina = medina[medina['EUCS Control (2022)'].astype(str).str.strip() != ""]

print(f"Righe MEDINA dopo pulizia: {len(medina)} (rimosse righe vuote)")

# --- 2. PREPARAZIONE FILE EUCS (Il 'Child') ---
def get_parent_id(val):
    if pd.isna(val): return None
    val = str(val).strip()
    # Prende tutto ciò che c'è prima del primo punto
    return val.split('.')[0]

# Creiamo la chiave per il raggruppamento
eucs['ParentKey'] = eucs['baseId'].apply(get_parent_id)

# Formattiamo il testo in modo leggibile
eucs['Formatted_Text'] = (
    "[" + eucs['baseId'].astype(str) + "] " + 
    "(" + eucs['assurance_level'].astype(str) + ") " + 
    eucs['description'].astype(str)
)

# RAGGRUPPAMENTO
eucs_grouped = eucs.groupby('ParentKey')['Formatted_Text'].apply(lambda x: "\n\n".join(x)).reset_index()
eucs_grouped.rename(columns={'Formatted_Text': 'Full_EUCS_Text'}, inplace=True)

print(f"Requisiti raggruppati in {len(eucs_grouped)} controlli principali.")

# --- 3. PREPARAZIONE FILE MEDINA (Il 'Parent') ---
def extract_mapping_id(val):
    if pd.isna(val): return None
    match = re.search(r'([A-Z]{2,4}-\d{2})', str(val))
    if match:
        return match.group(1)
    return None

medina['JoinKey'] = medina['EUCS Control (2022)'].apply(extract_mapping_id)

# --- 4. ESECUZIONE DEL MERGE ---
medina_final = pd.merge(
    medina,
    eucs_grouped,
    left_on='JoinKey',
    right_on='ParentKey',
    how='left'
)

# --- 5. PULIZIA E SALVATAGGIO ---
medina_final['EUCS Text'] = medina_final['Full_EUCS_Text'].fillna("Testo non trovato nel PDF estratto.")

# Rimuoviamo colonne temporanee
cols_to_drop = ['JoinKey', 'ParentKey', 'Full_EUCS_Text']
medina_final.drop(columns=[c for c in cols_to_drop if c in medina_final.columns], inplace=True)

# Salvataggio
output_filename = "Schemes/NewEucsRequirements.csv"
medina_final.to_csv(output_filename, index=False)

print("-" * 30)
print(f"✅ Merge completato senza righe vuote!")
print(f"File salvato in: {output_filename}")

# --- 6. CONTROLLO QUALITÀ RAPIDO ---
matches = medina_final[medina_final['EUCS Text'] != "Testo non trovato nel PDF estratto."]
print(f"📊 Successo: {len(matches)} su {len(medina_final)} righe hanno ora il testo.")

--- INIZIO MERGE (VERSIONE PULITA) ---
Righe originali EUCS: 467
Righe originali MEDINA: 120
Righe MEDINA dopo pulizia: 119 (rimosse righe vuote)
Requisiti raggruppati in 122 controlli principali.
------------------------------
✅ Merge completato senza righe vuote!
File salvato in: Schemes/NewEucsRequirements.csv
📊 Successo: 119 su 119 righe hanno ora il testo.


In [32]:
import pandas as pd

# --- CONFIGURAZIONE ---
file_path = "Schemes/NewEucsRequirements.csv"
missing_placeholder = "Testo non trovato nel PDF estratto."

print("--- CONTROLLO QUALITÀ DATASET FINALE ---\n")

try:
    df = pd.read_csv(file_path)
    
    # 1. ANALISI DEI MANCANTI (I 4 "Ribelli")
    missing_rows = df[df['EUCS Text'] == missing_placeholder]
    
    print(f"❌ CONTROLLI MANCANTI ({len(missing_rows)}):")
    if not missing_rows.empty:
        print(missing_rows[['EUCS Category', 'EUCS Control (2022)']].to_string(index=False))
        
        # Check automatico per Typo comuni
        ids_missing = missing_rows['EUCS Control (2022)'].astype(str).tolist()
        print("\n🔍 DIAGNOSI RAPIDA SUI MANCANTI:")
        for miss_id in ids_missing:
            if "OSI" in miss_id:
                print(f"  -> '{miss_id}': Sembra un errore di battitura (OSI invece di OIS). Controlla il CSV originale.")
            elif " " in miss_id.strip():
                 print(f"  -> '{miss_id}': Attenzione agli spazi extra o formattazione strana.")
    else:
        print("  Nessuno! (Wow!)")

    print("-" * 50)

    # 2. ANALISI LUNGHEZZA TESTO (Caccia ai "Falsi Positivi")
    # Escludiamo i mancanti
    valid_rows = df[df['EUCS Text'] != missing_placeholder].copy()
    valid_rows['Text_Length'] = valid_rows['EUCS Text'].str.len()
    
    # Ordiniamo per lunghezza (i più corti sono sospetti)
    shortest = valid_rows.sort_values('Text_Length').head(5)
    
    print(f"\n⚠️ CONTROLLI CON TESTO SOSPETTOSAMENTE CORTO (< 50 caratteri):")
    suspicious_short = shortest[shortest['Text_Length'] < 50]
    
    if not suspicious_short.empty:
        for idx, row in suspicious_short.iterrows():
            print(f"  - {row['EUCS Control (2022)']}: \"{row['EUCS Text']}\"")
    else:
        print("  Tutti i testi sembrano avere una lunghezza ragionevole (nessun artefatto vuoto).")

    print("-" * 50)

    # 3. SAMPLE VISIVO (Guardiamo in faccia i dati)
    print("\n📝 ESEMPIO CASUALE DI SUCCESSO:")
    if not valid_rows.empty:
        sample = valid_rows.sample(1).iloc[0]
        print(f"ID: {sample['EUCS Control (2022)']}")
        print(f"Testo (Anteprima):\n---\n{sample['EUCS Text'][:300]}...\n---")

except FileNotFoundError:
    print("Errore: File non trovato. Hai eseguito il merge nella cella precedente?")

--- CONTROLLO QUALITÀ DATASET FINALE ---

❌ CONTROLLI MANCANTI (0):
  Nessuno! (Wow!)
--------------------------------------------------

⚠️ CONTROLLI CON TESTO SOSPETTOSAMENTE CORTO (< 50 caratteri):
  Tutti i testi sembrano avere una lunghezza ragionevole (nessun artefatto vuoto).
--------------------------------------------------

📝 ESEMPIO CASUALE DI SUCCESSO:
ID: DOC-04 - GUIDELINES AND RECOMMENDATIONS FOR COMPOSITION
Testo (Anteprima):
---
[DOC-04] (Basic) The CSP shall provide a justification for the assurance level targeted in the certification, based on the risks associated to the cloud service’s targeted users and use cases

[DOC-04] (Basic) If the CSP claims compliance to security profiles for its cloud service, the justification...
---
